## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy
# Import API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")

vacation_df.head()


,Unnamed: 0,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Vila Velha,BR,300.12,scattered clouds,-20.3297,-40.2925,Quality Suites Vila Velha
1,8,Butaritari,KI,300.91,clear sky,3.0707,172.7902,Isles Sunset Lodge
2,14,Hithadhoo,MV,301.50,few clouds,-0.6000,73.0833,Scoop Guest House
3,21,Isangel,VU,301.21,broken clouds,-19.5500,169.2667,Tanna Lodge
4,31,Jatibarang,ID,302.99,clear sky,-6.4747,108.3153,Adi Sederhana Hotel


In [6]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dt>{Hotel Name}</dt>
<dt><b>City</b></dt><dt>{City}</dt>
<dt><b>Country</b></dt><dt>{Country}</dt>
<dt><b>Current Description</b></dt><dt>{Current Description}</dt>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
vacation_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]
locations

,Lat,Lng
0,-20.3297,-40.2925
1,3.0707,172.7902
2,-0.6000,73.0833
3,-19.5500,169.2667
4,-6.4747,108.3153
...,...,...
102,13.9833,-14.8000
103,24.7000,-77.7667
104,9.0333,-79.5000
105,21.4447,-157.8378


In [4]:
# 4a. Add a marker layer for each city to the map.
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=vacation_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

start = vacation_df.loc[vacation_df["City"] == "La Palma"]
end = vacation_df.loc[vacation_df["City"] == "La Palma"]
stop1 =  vacation_df.loc[vacation_df["City"] == "San Angelo"]
stop2 =  vacation_df.loc[vacation_df["City"] == "Ensley"]
stop3 =  vacation_df.loc[vacation_df["City"] == "Lumberton"] 
start

,Unnamed: 0,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name


In [22]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.

start_city = start["Lat"].to_numpy()[0],start["Lng"].to_numpy()[0]
end_city = end["Lat"].to_numpy()[0],end["Lng"].to_numpy()[0]
cood_city1 = stop1["Lat"].to_numpy()[0],stop1["Lng"].to_numpy()[0]
cood_city2= stop2["Lat"].to_numpy()[0],stop2["Lng"].to_numpy()[0]
cood_city3= stop3["Lat"].to_numpy()[0],stop3["Lng"].to_numpy()[0]
print(f'start_city : {start_city},end_city : {end_city}')
print(f'cood_city2 :{cood_city1},cood_city2 : {cood_city2},cood_city3 : {cood_city3}')      


start_city : (33.8464, -118.0467),end_city : (33.8464, -118.0467)
cood_city2 :(31.4638, -100.437),cood_city2 : (30.5188, -87.2728),cood_city3 : (34.6182, -79.0086)


In [23]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
city_itinerary = gmaps.directions_layer(
      start_city,end_city,waypoints = [cood_city1,cood_city2,cood_city3],travel_mode='DRIVING')     
fig.add_layer(city_itinerary) 
fig      


Figure(layout=FigureLayout(height='420px'))

In [24]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([start,end,stop1,stop2,stop3],ignore_index=True)
itinerary_df

,Unnamed: 0,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,350,La Palma,US,300.74,clear sky,33.8464,-118.0467,La Quinta Inn & Suites by Wyndham Buena Park
1,350,La Palma,US,300.74,clear sky,33.8464,-118.0467,La Quinta Inn & Suites by Wyndham Buena Park
2,599,San Angelo,US,302.95,few clouds,31.4638,-100.4370,"Motel 6 San Angelo, TX"
3,544,Ensley,US,301.21,clear sky,30.5188,-87.2728,Holiday Inn Express & Suites Pensacola W I-10
4,172,Lumberton,US,300.20,broken clouds,34.6182,-79.0086,"Country Inn & Suites by Radisson, Lumberton, NC"


In [27]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dt>{Hotel Name}</dt>
<dt><b>City</b></dt><dt>{City}</dt>
<dt><b>Country</b></dt><dt>{Country}</dt>
<dt><b>Current Description</b></dt><dt>{Current Description} and {Max Temp} °F</dt>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
itinerary_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map.
locations = itinerary_df[["Lat", "Lng"]]
max_temp = itinerary_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=itinerary_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))